In [1]:
import numpy as np
import pandas as pd
import os, sys

import json

import urllib.parse
from urllib.parse import unquote
from urllib.parse import urlparse

# Подготовка справочника от Yandex

In [2]:
%%time
theLongColumns = ['Название ресурса', 'Тип ресурса', 'Медиахолдинг', 'Данные Метрики',\
       'Посетители (кросс-девайс)', 'Посетители (браузер)', 'Среднее время',\
       'Доля пользователей приложения', 'Дневная аудитория']

#Процедура
def toSubject( szSubject ):
    nPoint = -1 if szSubject is None else szSubject.find('->')
    return szSubject.strip().lower() if nPoint == -1 else (szSubject[:nPoint]).strip().lower()

#Процедура
def workupSubject( szField ):
    return set([toSubject(value)  for value in szField.split(',') ])
#Процедура
def unboundedSubject( theCursor, theSubjectSet ) :
    arr = np.array(list(theSubjectSet))
    return arr[theCursor] if len(arr) > theCursor else np.nan

#Читаем скаченный справочник, удаляем лишние
theRadar = pd.read_csv('~/project/radar.csv', sep=',' )
theRadar['Тематики ресурса'] = theRadar['Тематики ресурса'].fillna(theRadar['Тип ресурса'])
theRadar.drop(theLongColumns, axis=1, inplace=True)
theRadar.rename({'URL-адрес ресурса':'domain', 'Тематики ресурса':'subject'}, axis=1, inplace=True )

#Перебираем элементы, и отрезаем все лишнее
theRadar['subject'] = theRadar['subject'].apply( workupSubject )

#Раскладываем перечисление на колонки
for cursor in range(0, 7):
    theRadar['subject' + str(cursor)] = theRadar['subject'].apply( lambda x: unboundedSubject(cursor,  x ) )
    
theRadar.drop(['subject'], axis=1, inplace=True)
theRadar.rename({'subject0':'subject'}, axis=1, inplace=True )

 #Объединяем темы, по используемым
theRadar['subject1'].fillna( '', inplace=True )
theRadar['subject2'].fillna( '', inplace=True )
theRadar['subject'] = theRadar[['subject', 'subject1', 'subject2']].agg(' '.join, axis=1)#

#Удаляем неиспользуемое
theRadar.drop(['subject1', 'subject2', 'subject3', 'subject4', 'subject5', 'subject6'], axis=1, inplace=True)
#theRadar.shape, theRadar.info()
#theRadar.head(25)
theRadar.to_csv('~/subject.csv', sep=',', index=False)

CPU times: user 632 ms, sys: 48 ms, total: 680 ms
Wall time: 687 ms


# Чистка URL для BagOfWords

In [ ]:
sourceName = '~/NoTargetData.csv'
#sourceName = '/data/share/project01/gender_age_dataset.txt'

In [ ]:
%%time
#Процедура. Фильтрует домен из url
def toDomain( url ):
    if url.startswith('http://http') : url = url[7:]
    if url.startswith('http://&referrer=') : url = url[17:]
        
    parsed_url = urlparse( urllib.parse.unquote( url ).strip() )
    if parsed_url.scheme not in ['http','https']: return None

    url = parsed_url.netloc.strip()

    if url.startswith('www.') : url = url[4:]

    dpoint = url.rfind(':')     
    if dpoint != -1 : url = url[:dpoint]    

    dpoint = url.find('&')     
    if dpoint != -1 : url = url[:dpoint]    

    dpoint = url.rfind('@')     
    if dpoint != -1 : url = url[dpoint+1:]    
       
    if url.rfind('.') == -1: return None

    theSplit = url.split('.')
    return url if len(theSplit) < 3 else theSplit[-2] + '.' + theSplit[-1]

def toSubject(domain):
    try: return theRadar.loc[domain]['subject'].strip()  
    except: return 'прочее'
    
#Генерируем фичи для uid
def toFetch( row ):
    theUidList = [ toDomain(value['url']) for value in json.loads( row[1].user_json) ['visits'] ]
    theUidList = [toSubject(value) for value in theUidList if value is not None]
    dict_ = {'uid':row[1].uid, 'text':' '.join( theUidList )}
    return dict_
    
#Перебираем элементы, сохраняя из данных тока домен 
theRadar = pd.read_csv('~/subject.csv', sep=',' )
theRadar.set_index(['domain'], inplace=True)
theRawData = pd.read_csv(sourceName, sep='\t' )
theFeatch =  pd.DataFrame( [toFetch( row ) for row in theRawData.iterrows()] )
theFeatch.set_index(['uid'], inplace=True)

In [ ]:
theFeatch.head()
#theFeatch.shape

# Чистка URL для Классификатора

In [3]:
#sourceName = '~/NoTargetData.csv'
sourceName = '/data/share/project01/gender_age_dataset.txt'

In [4]:
%%time
#Процедура. Фильтрует домен из url
def toDomain( url ):
    if url.startswith('http://http') : url = url[7:]
    if url.startswith('http://&referrer=') : url = url[17:]
        
    parsed_url = urlparse( urllib.parse.unquote( url ).strip() )
    if parsed_url.scheme not in ['http','https']: return None

    url = parsed_url.netloc.strip()

    if url.startswith('www.') : url = url[4:]

    dpoint = url.rfind(':')     
    if dpoint != -1 : url = url[:dpoint]    

    dpoint = url.find('&')     
    if dpoint != -1 : url = url[:dpoint]    

    dpoint = url.rfind('@')     
    if dpoint != -1 : url = url[dpoint+1:]    
       
    if url.rfind('.') == -1: return None

    theSplit = url.split('.')
    return url if len(theSplit) < 3 else theSplit[-2] + '.' + theSplit[-1]

def toSubject(domain):
    try: return theRadar.loc[domain]['subject'].strip()  
    except: return 'прочее'
    
#Генерируем фичи для uid
def toFetch( row ):
    try:
        theJson = json.loads( row[1].user_json)
        theUidList = [ toDomain(value['url']) for value in  theJson['visits'] ]
        theUidList = [value for value in theUidList if value is not None]
    except : 
        theUidList = ['nodata.ru']
        
    theUid = pd.DataFrame( 0, index = theUidList, columns = ['footprint'] )
    theUid.index.name = 'domain'
    theUid = theUid.groupby(['domain']).count()
    theUid['subject'] = theUid.apply(lambda value : toSubject(value.name), axis = 1 )

    dict_ = {'uid':row[1].uid }
    for _, theLine in theUid.iterrows():
        dict_[ theLine['subject'] ] = theLine['footprint'] 
    return dict_

    
#Перебираем элементы, сохраняя из данных тока домен 
theRadar = pd.read_csv('~/subject.csv', sep=',' )
theRadar.set_index(['domain'], inplace=True)
theRawData = pd.read_csv(sourceName, sep='\t' )
theFeatch =  pd.DataFrame( [toFetch( row ) for row in theRawData.iterrows()] )
theFeatch.set_index(['uid'], inplace=True)

CPU times: user 6min 21s, sys: 11.8 s, total: 6min 32s
Wall time: 6min 36s


In [5]:
#theFeatch.head()
theFeatch.shape

(41138, 96)

# Формируем полную таблицу для исследований


In [6]:
%%time
theRawData.drop(['user_json'], axis=1, inplace=True)
theRawData.set_index(['uid'], inplace=True)

theFeatch = pd.merge(theRawData, theFeatch, on='uid', how = 'left')
theFeatch.reset_index(inplace=True)
theFeatch.to_csv('~/project/fetch3S.csv', sep=',', index=False)

CPU times: user 5.89 s, sys: 404 ms, total: 6.3 s
Wall time: 4.58 s
